# 볼린저 밴드 함수화
1. 첫번쨰 함수를 생성(매개변수 데이터프레임 하나)
    1. 결측치와 이상치를 제거한다.
    2. 컬럼을 종가만 남기고 모두 삭제
    3. 이동평균선, 상단밴드, 하단밴드 파생변수 생성
2. 두번쨰 함수를 생성(매개변수 데이터프레임 하나)
    1. 거래 내역이라는 파생변수 생성 값은 ''
    2. 볼린저 밴드의 거래 내역을 추가
3. 세번째 함수를 생성(매개변수 데이터프레임 하나)
    1. 수익율 파생변수 생성 값은 '1'
    2. 판매를 한 날의 수익률 변경
    3. 누적 수익율을 계산하여 새로운 파생변수를 생성
    4. 최종 누적 수익율을 출력(print)

In [101]:
import pandas as pd
import numpy as np
from datetime import datetime

In [115]:
# 1번 함수 생성
# 매개 변수 추가 
def create_band(df, col, start, end):
    # 인덱스를 시계열로 변경
    df.index = pd.to_datetime(df.index)
    # start, end를 시계열로 변경
    start = datetime.strptime(start, '%Y%m%d').isoformat()
    end = datetime.strptime(end, '%Y%m%d').isoformat()
    # 데이터를 시작시간부터 종료 시간까지 필터
    df = df.loc[start:end]
    # 결측치와 이상치를 제거 
    df = df.loc[~df.isin([np.nan, np.inf, -np.inf]).any(axis='columns'), [col]]
    # 수정 종가 컬럼을 제외한 데이터프레임 생성
    # df = df[[col]]
    # 이동 평균선 생성
    df['center'] = df.rolling(20).mean()
    # 상단 밴드 생성
    df['ub'] = df['center'] + ( 2 * df[col].rolling(20).std() )
    # 하단 밴드 생성
    df['lb'] = df['center'] - ( 2 * df[col].rolling(20).std() )
    # 결과를 리턴
    return df

In [116]:
load_data = pd.read_csv("../csv/GM.csv", index_col='Date')

In [114]:
data = create_band(load_data, 'Close', '20110101', '20150101')

In [129]:
data.head(1)

,Close,center,ub,lb,trade,return,acc_rtn
Date,,,,,,,
2011-01-03,37.060001,NaN,NaN,NaN,,1.0,1.0


In [118]:
# 2번 함수 생성
def add_trade (df, col):
    col = df.columns[0]
    # 기준이 되는 컬럼은 무엇인가?
    # 기준이 되는 컬럼은 컬럼 중에 첫번째이기 때문에 df.columns[0]
    # trade 파생변수 생성
    df['trade'] = ''
    # 거래 내역을 추가
    for i in df.index:
    # 상단 밴드보다 종가가 높은 경우
        if df.loc[i, col] > df.loc[i, 'ub']:
            # 현재 구매 상태이면
            if df.shift(1).loc[i, 'trade'] == 'buy':
                # 매도
                df.loc[i, 'trade'] = ''
            else:
                df.loc[i, 'trade'] = ''
        # 하단 밴드보다 종가가 낮은 경우
        elif df.loc[i, col] < df.loc[i, 'lb']:
            # 현재 구매 상태이면
            if df.shift(1).loc[i, 'trade'] == 'buy':
                # 구매 상태를 유지
                df.loc[i, 'trade'] = 'buy'
            else:
                # 매수
                df.loc[i, 'trade'] = 'buy' 
        else:
            # 현재 구매 상태이면
            if df.shift(1).loc[i, 'trade'] == 'buy':
                # 구매 상태를 유지
                df.loc[i, 'trade'] = 'buy'
            else:
                df.loc[i, 'trade'] = ''
    return df

In [133]:
add_trade(data, 'close')['trade'].value_counts()

buy    559
       447
Name: trade, dtype: int64

In [134]:
data = add_trade(data, 'Close')

In [142]:
# 3번째 함수 생성
def add_rtn(df):
    col = df.columns[0]
    # 수익율 파생변수 생성
    df['return'] = 1
    # 판매한 날의 수익율 대입
    rtn = 1.0
    buy = 0.0
    sell = 0.0

    for i in df.index:
        # 구매가를 출력
        if (df.shift(1).loc[i, 'trade'] == '') and \
            (df.loc[i, 'trade'] == 'buy'):
            buy = df.loc[i, col]
            print('진입일 :', i, '구매 가격 :', buy)
        # 판매가를 출력
        elif (df.shift(1).loc[i, 'trade'] == 'buy') and \
            (df.loc[i, 'trade'] == ''):
            sell = df.loc[i, col]
            rtn = (sell - buy) / buy + 1
            df.loc[i, 'return'] = rtn
            print('판매일 :', i, '판매 가격 :', sell, '수익율 :', rtn)

        # 구매가, 판매가를 초기화
        if df.loc[i, 'trade'] == '':
            buy = 0.0
            sell = 0.0
    
    # 누적 수익율 계산하여 파생변수에 대입
    acc_rtn = 1.0

    for i in df.index:
        rtn = df.loc[i, 'return']
        acc_rtn *= rtn
        df.loc[i, 'acc_rtn'] = acc_rtn

    print('누적 수익율 :', acc_rtn)

    return df

In [143]:
add_rtn(data)

진입일 : 2011-02-02 00:00:00 구매 가격 : 35.68
판매일 : 2011-04-05 00:00:00 판매 가격 : 32.869999 수익율 : 0.9212443665919282
진입일 : 2011-06-02 00:00:00 구매 가격 : 29.6
판매일 : 2011-07-07 00:00:00 판매 가격 : 31.799999 수익율 : 1.0743242905405406
진입일 : 2011-07-27 00:00:00 구매 가격 : 28.139999
판매일 : 2012-01-05 00:00:00 판매 가격 : 22.17 수익율 : 0.7878465098737211
진입일 : 2012-03-06 00:00:00 구매 가격 : 24.58
판매일 : 2012-04-02 00:00:00 판매 가격 : 26.76 수익율 : 1.0886899918633037
진입일 : 2012-04-09 00:00:00 구매 가격 : 24.200001
판매일 : 2012-08-07 00:00:00 판매 가격 : 20.42 수익율 : 0.8438016180247265
진입일 : 2013-02-20 00:00:00 구매 가격 : 27.1
판매일 : 2013-03-20 00:00:00 판매 가격 : 29.200001 수익율 : 1.0774908118081181
진입일 : 2013-06-20 00:00:00 구매 가격 : 32.560001
판매일 : 2013-09-10 00:00:00 판매 가격 : 37.0 수익율 : 1.1363636014630343
진입일 : 2013-10-03 00:00:00 구매 가격 : 35.290001
판매일 : 2013-10-30 00:00:00 판매 가격 : 37.23 수익율 : 1.0549730502982984
진입일 : 2014-01-16 00:00:00 구매 가격 : 39.0
판매일 : 2014-03-05 00:00:00 판매 가격 : 37.52 수익율 : 0.9620512820512821
진입일 : 2014-03-13 00:00:00 구매 가격

,Close,center,ub,lb,trade,return,acc_rtn
Date,,,,,,,
2011-01-03,37.060001,NaN,NaN,NaN,,1.0,1.000000
2011-01-04,37.900002,NaN,NaN,NaN,,1.0,1.000000
2011-01-05,38.070000,NaN,NaN,NaN,,1.0,1.000000
2011-01-06,38.900002,NaN,NaN,NaN,,1.0,1.000000
2011-01-07,38.980000,NaN,NaN,NaN,,1.0,1.000000
...,...,...,...,...,...,...,...
2014-12-24,33.430000,32.5620,34.464492,30.659507,buy,1.0,0.862091
2014-12-26,33.730000,32.6450,34.601199,30.688801,buy,1.0,0.862091
2014-12-29,34.599998,32.7035,34.821803,30.585196,buy,1.0,0.862091


In [167]:
import bollinger as bol
import imp
imp.reload(bol)

<module 'bollinger' (<_frozen_importlib_external.NamespaceLoader object at 0x1187906d0>)>

In [168]:
# 클래스 생성
data = pd.read_csv('../csv/AAPL.csv', index_col='Date')

In [169]:
class_a = bg.Invest(data, 'Adj Close')

AttributeError: module 'bollinger' has no attribute 'Invest'